In [ ]:
import numpy as np
import keras
import keras.backend as kb
import tensorflow as tf
import pandas as pd


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.layers import Dense
from sklearn.model_selection import train_test_split
# from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv', sep=',',header=None )
# Need sampling? data should be sorted and grouped by label
# df = df.sample(frac=1)
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       150 non-null    float64
 1   1       150 non-null    float64
 2   2       150 non-null    float64
 3   3       150 non-null    float64
 4   4       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
df.describe()

,0,1,2,3
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [ ]:
df.corr()

,0,1,2,3
0,1.000000,-0.109369,0.871754,0.817954
1,-0.109369,1.000000,-0.420516,-0.356544
2,0.871754,-0.420516,1.000000,0.962757
3,0.817954,-0.356544,0.962757,1.000000


In [ ]:
df_values =df.values
x = df_values[:,:4].astype(float)
y =df_values[:,-1]
x.shape

(150, 4)

In [ ]:
#since the out Y is a string, it is always prefered to to convert it into form of binary categorical vector (one-hot-vector)
# that is convert the Y value to 1 and 0 for each class
# this done by using scikit learn Label encoder function, this function encodes the Y values to interger 
# note : prerequisite - data should be sorted by label or encoding fails

encoder = LabelEncoder()
encoded_y = encoder.fit_transform(y)
encoded_Y = np_utils.to_categorical(encoded_y).astype(float)

encoded_Y.shape

(150, 3)

In [ ]:
# split into train vs test dataset

X_train, X_test, Y_train , Y_test = train_test_split(x,encoded_Y , test_size = 0.20 , random_state =24 , shuffle = True)

In [ ]:
# build model

model = keras.Sequential()
model.add(Dense(10,input_dim=X_train.shape[1],activation=tf.nn.relu))
model.add(Dense(5,activation=tf.nn.relu))
model.add(Dense(3,activation=tf.nn.softmax))

optimizer = tf.keras.optimizers.Adam(0.003)
model.compile(optimizer=optimizer,
              loss ='categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 18        
Total params: 123
Trainable params: 123
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit( X_train, Y_train, epochs=1000 ,verbose=2, batch_size=30)

Epoch 1/1000
4/4 - 0s - loss: 1.2309 - categorical_accuracy: 0.6167
Epoch 2/1000
4/4 - 0s - loss: 1.1024 - categorical_accuracy: 0.6167
Epoch 3/1000
4/4 - 0s - loss: 1.0279 - categorical_accuracy: 0.6250
Epoch 4/1000
4/4 - 0s - loss: 0.9787 - categorical_accuracy: 0.6667
Epoch 5/1000
4/4 - 0s - loss: 0.9389 - categorical_accuracy: 0.8167
Epoch 6/1000
4/4 - 0s - loss: 0.8933 - categorical_accuracy: 0.8667
Epoch 7/1000
4/4 - 0s - loss: 0.8497 - categorical_accuracy: 0.7917
Epoch 8/1000
4/4 - 0s - loss: 0.8297 - categorical_accuracy: 0.8333
Epoch 9/1000
4/4 - 0s - loss: 0.7987 - categorical_accuracy: 0.8417
Epoch 10/1000
4/4 - 0s - loss: 0.7642 - categorical_accuracy: 0.8333
Epoch 11/1000
4/4 - 0s - loss: 0.7273 - categorical_accuracy: 0.8500
Epoch 12/1000
4/4 - 0s - loss: 0.6926 - categorical_accuracy: 0.9000
Epoch 13/1000
4/4 - 0s - loss: 0.6575 - categorical_accuracy: 0.9250
Epoch 14/1000
4/4 - 0s - loss: 0.6239 - categorical_accuracy: 0.9083
Epoch 15/1000
4/4 - 0s - loss: 0.5965 - cat

In [ ]:
res = model.evaluate(X_test,Y_test, batch_size = 30)

print("result - {}".format(res))

1/1 [==============================] - 0s 111ms/step - loss: 0.0279 - categorical_accuracy: 0.9667
result - [0.027919916436076164, 0.9666666388511658]


In [ ]:
predictions = model.predict(X_test[:3])
print(np.round(predictions))

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [ ]:
# training with feature selection
pf = df.copy()

In [ ]:
pf.corr()

,0,1,2,3
0,1.000000,-0.109369,0.871754,0.817954
1,-0.109369,1.000000,-0.420516,-0.356544
2,0.871754,-0.420516,1.000000,0.962757
3,0.817954,-0.356544,0.962757,1.000000


In [ ]:
pf.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
#feature 2 and 2 have high correlation
# lets try to train with this and check accuracy
features = pf[[2,3]]
feature_target = encoded_Y.copy()

In [ ]:
print(features.shape, feature_target.shape)

(150, 2) (150, 3)


In [ ]:
nx_train,nx_test, ny_train,nytest = train_test_split(features,feature_target,test_size = 0.20, random_state=12)
print(nx_train.shape,nx_test.shape)
print(ny_train.shape,nytest.shape)

(120, 2) (30, 2)
(120, 3) (30, 3)


In [ ]:
#build model
feature_model = keras.Sequential([
                                  Dense(10, input_dim=features.shape[1], activation=tf.nn.relu6),
                                  Dense(5, activation=tf.nn.relu),
                                  Dense(3, activation=tf.nn.softmax)
                                  ]
                                  )
optimizer = tf.keras.optimizers.Adam(0.003)
feature_model.compile(optimizer=optimizer,
              loss ='categorical_crossentropy', 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [ ]:
feature_model.fit( nx_train, ny_train, epochs=1000 ,verbose=2, batch_size=30)

Epoch 1/1000
4/4 - 0s - loss: 3.7803 - categorical_accuracy: 0.3417
Epoch 2/1000
4/4 - 0s - loss: 3.4049 - categorical_accuracy: 0.3417
Epoch 3/1000
4/4 - 0s - loss: 3.0753 - categorical_accuracy: 0.3417
Epoch 4/1000
4/4 - 0s - loss: 2.7404 - categorical_accuracy: 0.3417
Epoch 5/1000
4/4 - 0s - loss: 2.4672 - categorical_accuracy: 0.3417
Epoch 6/1000
4/4 - 0s - loss: 2.2282 - categorical_accuracy: 0.3417
Epoch 7/1000
4/4 - 0s - loss: 2.0133 - categorical_accuracy: 0.3417
Epoch 8/1000
4/4 - 0s - loss: 1.8128 - categorical_accuracy: 0.3417
Epoch 9/1000
4/4 - 0s - loss: 1.6498 - categorical_accuracy: 0.3583
Epoch 10/1000
4/4 - 0s - loss: 1.5112 - categorical_accuracy: 0.4083
Epoch 11/1000
4/4 - 0s - loss: 1.3840 - categorical_accuracy: 0.5333
Epoch 12/1000
4/4 - 0s - loss: 1.2615 - categorical_accuracy: 0.6000
Epoch 13/1000
4/4 - 0s - loss: 1.1757 - categorical_accuracy: 0.6333
Epoch 14/1000
4/4 - 0s - loss: 1.0863 - categorical_accuracy: 0.6417
Epoch 15/1000
4/4 - 0s - loss: 1.0209 - cat

In [ ]:
res=feature_model.evaluate(nx_test,nytest, batch_size = 30)
res

1/1 [==============================] - 0s 105ms/step - loss: 0.0833 - categorical_accuracy: 0.9667


[0.08326780050992966, 0.9666666388511658]

In [ ]:
pred = feature_model.predict(nx_test[:3])
print(np.round(predictions))

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
